# Visualization of Torsiondrive 1D scan result as scatter plot and interactively showing molecular structures.


## Specify your scan.xyz (produced by torsiondrive) and scan.pdb (converted from scan.xyz by Molecule.py) below.

In [15]:
scanxyz = "/home/winter/onsager/tsqm/td/1dscan.xyz"
scanpdb = "/home/winter/onsager/tsqm/td/1dscan.pdb"

## Execute the following cell to see an interactive plot. Click on the scan point to corresponding show structure.

In [17]:
import numpy as np
import nglview as nv
from IPython.display import display
import plotly.graph_objs as go
import plotly

def show_structure_for_click(trace, point, selector):
    x=point.xs[0]
    print ("Showing optimized conformation for x = " +str(x))
    # pick the lines correponding to x in scan.pdb and write a temporary pdb for visualization.
    position = int(1 + (x-xmin)//xstep)
    with open(scanpdb) as f:
        lines = f.readlines()
    wanted_lines = lines[1+(n_atoms+3)*(position-1):1+(n_atoms+3)*position]
    with open('temp.pdb', 'w') as f:
        for item in wanted_lines:
            f.write("%s" % item)
    view = nv.show_structure_file("temp.pdb")
    view.background = 'white'
    display(view)

# Parse dihedrals and energies. From Yudong's script.
with open(scanxyz) as f:
    lines = f.readlines()
global n_atoms 
n_atoms = int(lines[0])
comment_lines = lines[1::n_atoms+2]
grid_data = dict()
for line in comment_lines:
    ls = line.strip().split()
    assert ls[0] == 'Dihedral' and ls[-2] == 'Energy', line
    grid_energy = float(ls[-1])
    grid_coord = []
    for i in range(1, len(ls) - 2):
        c = int(ls[i].replace('(', '').replace(',','').replace(')',''))
        grid_coord.append(c)
    grid_data[tuple(grid_coord)] = grid_energy
    
# auto determination of No. of scan points (xpts), minimum values (xmin) and step size (xstep)
global xpts, xmin, xstep
xpts = len(grid_data)
xmin = float(lines[1].strip().split()[1][1:-2])
xstep = float(lines[1+n_atoms+2].strip().split()[1][1:-2]) - float(lines[1].strip().split()[1][1:-2])

x = np.linspace(xmin,xmin+xstep*(xpts-1),num=xpts)
rawe = np.fromiter(grid_data.values(), dtype=float).T
# convering rawe in hartree to e in kcal/mol with lowest value offset to zero.
e = rawe*627.509-np.min(rawe*627.509)

# Make the plot.  
fig = go.FigureWidget([go.Scatter(x=x, y=e)])
#There is a glitch rendering figure widget if go.Layout(...) is added to go.FigureWidget(). 
#For custom layout, use the cell below that renders static figure.

fig.data[0].on_click(show_structure_for_click)
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'eadfa30e-0813-4c2d-a5b8-f86dbcf1e772',
 …

## Too many structures? Re-execute the above cell to clear them.

## Wanna see a static energy surface? Execute the cell below.

In [18]:
plotly.offline.init_notebook_mode(connected=True)
static = go.Figure([go.Scatter(x=x, y=e)],
                   go.Layout(xaxis=dict(tickvals=x, tickangle=30),
                             yaxis=dict(title='Relative Energy (kcal/mol)')))
plotly.offline.iplot(static)

## Known issues: this notebook doesn't work if opened in jupyterlab, so open it by vanilla jupyter notebook.